
#### OBJETIVO: Leer todos los archivos de las variables que estan en diferentes formatos y generar datos tiff  para luego hacer los stacks diarios y poder hacer las predicciones de PM2.5

#

In [ ]:
import os
import pandas as pd
import xarray as xr
import rioxarray as rxr
import rasterio
from functools import reduce
import matplotlib.pyplot as plt
import netCDF4
from rasterio.warp import reproject, Resampling
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.enums import Resampling as RioResampling
from rasterio.io import MemoryFile
# from osgeo import gdal
import rioxarray
from datetime import datetime
import os

import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import glob
import numpy as np

print("librerias ok")

librerias ok


In [16]:
#Datos iniciales
estacion = "CH"
numRaster =  "01"
ndvi = rioxarray.open_rasterio(
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/rasterTemplate/{numRaster}_raster_template.tif"
).squeeze()

year = 2015
mes = "01-06-"
ndvi

<xarray.DataArray (y: 33, x: 32)> Size: 4kB
[1056 values with dtype=float32]
Coordinates:
  * y            (y) float64 264B -33.33 -33.34 -33.34 ... -33.58 -33.59 -33.59
  * x            (x) float64 256B -70.82 -70.81 -70.8 ... -70.53 -70.52 -70.51
    band         int64 8B 1
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  0.56723296642303
    STATISTICS_MEAN:     -9999
    STATISTICS_MINIMUM:  0.12767471373081
    STATISTICS_STDDEV:   -9999
    _FillValue:          -3.4e+38
    scale_factor:        1.0
    add_offset:          0.0

In [ ]:
# -----------------------   01 MAIAC  ------------------------------
# No funciona porque no esta gdal para abrir los hdf, hay que hacerlo por ahora en r
estacion = "SP"

file = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/00_MAIAC/2010/01-06-2010/MCD19A2.A2010001.h13v11.061.2022327191334.hdf"

import h5py


with h5py.File(file, "r") as f:
    def print_tree(name, obj):
        print(name)
    f.visititems(print_tree)



OSError: Unable to synchronously open file (file signature not found)

In [ ]:
# -----------------------   01 NDVI  ------------------------------
# Igual que MAIAC  funciona porque no esta gdal para abrir los hdf, hay que hacerlo por ahora en r

In [ ]:
# -----------------------   05 ERA5   ------------------------------
# Setear datos de entrada
estacion = "SP"
year = "2010"
numRaster = "01"

# este es el raster template generado a partir del dato del ndvi.
#Esto se hizo antes en r
raster_template = rioxarray.open_rasterio(
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/rasterTemplate/{numRaster}_raster_template.tif"
).squeeze()


dir_era = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/"
    f"meteoSatelital/TP/{year}/"
)

dir_out = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/"
    f"dataset_ejemplo/Prediccion_{year}/tiff/05_ERA5_V01/"
)
# Ver si existe el directorio sino se genra
os.makedirs(dir_out, exist_ok=True)
######################################################################
# Procesamiento de los archivos en formato netcdf
# Vemos cuantos archivos hay
era_files = sorted(glob.glob(os.path.join(dir_era, "*.nc")))
era_files
print("Hay",len(era_files), "archivos de ERA5")

# Empieza el proncesamiento de todos los archivos
for i, era_file in enumerate(era_files, start=1):
    #print(i, era_file)

    nombre_era = os.path.basename(era_file)[:10]
    ds = xr.open_dataset(era_file)

    # seleccionar variable
    tp = ds["tp"]

    # promedio temporal (equivalente a calc(mean))
    tp_mean = tp.mean(dim="valid_time", skipna=True)
    #Cambiar si es necsario la proyeccion
    tp_mean = tp_mean.rio.write_crs("EPSG:4326", inplace=True)
    # Resamplear al tamaño del raster_template (1km)
    tp_resampled = tp_mean.rio.reproject_match(raster_template, resampling=Resampling.bilinear)
    #Recortar dominio
    tp_cropped = tp_resampled.rio.clip_box(*ndvi.rio.bounds())
    #Generar nombre de salida
    out_file = os.path.join(dir_out, f"{nombre_era}-TP_raster.tif")
    #Guardar archivo
    tp_cropped.rio.to_raster(out_file)


#Vemos si funciona
#print(tp_mean)

Hay 2 archivos de ERA5
<xarray.DataArray 'tp' (latitude: 4, longitude: 6)> Size: 96B
array([[9.1130105e-06, 2.2645001e-05, 4.4432731e-05, 5.4820874e-05,
        5.3691878e-05, 4.2216674e-05],
       [2.8765257e-07, 5.7855567e-07, 2.7723672e-06, 5.8600513e-06,
        6.7469168e-06, 6.1181167e-06],
       [2.8817365e-06, 1.8705468e-06, 3.0733111e-06, 6.2006711e-06,
        6.0006491e-06, 1.9839499e-06],
       [2.8428312e-06, 1.8225546e-06, 9.0635029e-07, 8.5082200e-07,
        7.0103732e-07, 3.1873000e-07]], dtype=float32)
Coordinates:
  * latitude     (latitude) float64 32B -23.25 -23.5 -23.75 -24.0
  * longitude    (longitude) float64 48B -47.14 -46.89 -46.64 ... -46.14 -45.89
    number       int64 8B 0
    spatial_ref  int64 8B 0
Attributes: (12/32)
    GRIB_paramId:                             228
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      24
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:               

In [ ]:
# -----------------------   06 MERRA-2   ------------------------------

# Datos de entrada del procesamiento
estacion = "SP"
year = 2010
numRaster = "01"

#Template para recortar el dominio de cada sitio
raster_template = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/"
    f"rasterTemplate/{numRaster}_raster_template.tif"
)
raster_template = rxr.open_rasterio(raster_template).squeeze()

#Direcotio donde se encuentran los originales
dir_merra = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/"
    f"04_MERRA-2_Dia/{year}/"
)
#Directorio para guardar el archivo recortado y resampleado
dir_out = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/"
    f"dataset_ejemplo/Prediccion_{year}/tiff/04_MERRA-2_Dia_V01/"
)

os.makedirs(dir_out, exist_ok=True)
#Vemos cuantos archivos hay
merra_files = sorted(glob.glob(os.path.join(dir_merra, "*.nc")))
print("Cantidad de archivos de MERRA-2:",len(merra_files))

#Variables descargadas de MERRA-2
variables_merra = [
    "BCSMASS",
    "DUSMASS",
    "SO2SMASS",
    "SO4SMASS",
    "SSSMASS",
]

###################################################################
#Procesamiento de la informacion de MERRA-2
for i, merra_file in enumerate(merra_files, start=1):
    print(i, merra_file)

    nombre_merra = os.path.basename(merra_file)[27:35]
    #Abrir archivo
    ds = xr.open_dataset(merra_file)
    #Recorrer la lista con todas las variables
    for var in variables_merra:
        if var not in ds:
            print(f"  {var} no está en el archivo")
            continue

        da = ds[var]

        # asegurar CRS
        da = da.rio.write_crs("EPSG:4326", inplace=True)

        # reproyectar + resamplear segun el tmplate
        da_match = da.rio.reproject_match(raster_template,resampling=Resampling.bilinear   )
        #Generar nombre del archivi
        out_file = os.path.join(dir_out,f"{nombre_merra}-{var}_raster.tif")

        da_match.rio.to_raster(out_file)




Cantidad de archivos de MERRA-2: 3
1 D:/Josefina/Proyectos/ProyectoChile/SP/dataset/04_MERRA-2_Dia/2010\MERRA2_300.tavg1_2d_aer_Nx.20100101.SUB.nc
2 D:/Josefina/Proyectos/ProyectoChile/SP/dataset/04_MERRA-2_Dia/2010\MERRA2_300.tavg1_2d_aer_Nx.20100102.SUB.nc
3 D:/Josefina/Proyectos/ProyectoChile/SP/dataset/04_MERRA-2_Dia/2010\MERRA2_300.tavg1_2d_aer_Nx.20100103.SUB.nc


In [ ]:
# -----------------------   06 AOD MERRA-2   ------------------------------

# Datos de entrada del procesamiento
estacion = "SP"
year = 2010
numRaster = "01"

#Template para recortar el dominio de cada sitio
raster_template = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/"
    f"rasterTemplate/{numRaster}_raster_template.tif"
)
raster_template = rxr.open_rasterio(raster_template).squeeze()

#Direcotio donde se encuentran los originales
dir_merra = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/"
    f"04_MERRA-2_Dia/{year}/"
)
#Directorio para guardar el archivo recortado y resampleado
dir_out = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/"
    f"dataset_ejemplo/Prediccion_{year}/tiff/07_MERRA-2_Dia_AOD_V01/"
)

os.makedirs(dir_out, exist_ok=True)

files = sorted(glob.glob(os.path.join(dir_merra, "*.nc")))
for i, nc_file in enumerate(files, start=1):
    print(i, nc_file)

    nombre_merra = os.path.basename(nc_file)[27:35]
    ds = xr.open_dataset(nc_file)
    da = list(ds.data_vars.values())[0]

    da = ds["AOD"]   # ajustar si el nombre cambia

    print(ds.data_vars)


    da = da.rio.write_crs("EPSG:4326", inplace=True)

    da_match = da.rio.reproject_match(
        ndvi,
        resampling=Resampling.bilinear
    )

    out_file = os.path.join(
        dir_out,
        f"{nombre_merra}-AOD-MERRA_raster.tif"
    )

    da_match.rio.to_raster(out_file)


In [ ]:
# -----------------------   06 AOD MERRA-2   ------------------------------

# Datos de entrada del procesamiento
estacion = "SP"
year = 2010
numRaster = "01"

#Template para recortar el dominio de cada sitio
raster_template = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/"
    f"rasterTemplate/{numRaster}_raster_template.tif"
)
raster_template = rxr.open_rasterio(raster_template).squeeze()

dir_era = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/dataset/"
    f"meteoSatelital/TP/{year}"
)

dir_out = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/"
    f"dataset_ejemplo/Prediccion_{year}/tiff/06_weekDay/"
)

os.makedirs(dir_out, exist_ok=True)

files = sorted(glob.glob(os.path.join(dir_era, "*.nc")))
print("Cantidad de archivos:",len(files))

for i, file in enumerate(files, start=1):
    print(i)
    nombre_weekDay = os.path.basename(file)[:10]  # YYYY-MM-DD

    fecha = datetime.strptime(nombre_weekDay, "%Y-%m-%d")

    # Lunes = 1, Domingo = 7 (equivalente a wday)
    day_week = fecha.weekday() + 1

    data = np.full(
    (ndvi.shape[0], ndvi.shape[1]),
    day_week,
    dtype=np.int16
)
    
    dayweek_raster = xr.DataArray(
        data,
        coords=ndvi.coords,
        dims=ndvi.dims
    )

    dayweek_raster = dayweek_raster.rio.write_crs(ndvi.rio.crs)
    dayweek_raster = xr.DataArray(
        data,
        coords=ndvi.coords,
        dims=ndvi.dims
    )

    dayweek_raster = dayweek_raster.rio.write_crs(ndvi.rio.crs)




Cantidad de archivos: 2
